In [1]:
!pip install flask

In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model_path = r"C:\Users\Vanshika Rana\Downloads\first_model.hdf5"
sec_model = load_model(model_path)

app = Flask(__name__)

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(350, 350))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Rescale to match the training data
    return img

def predict_class(img_path, model):
    # Preprocess the image
    img = preprocess_image(img_path)
    
    # Predict the class probabilities
    probabilities = model.predict(img)[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(probabilities)
    
    return predicted_class_index, probabilities

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})
    
    img_path = "temp.jpg"  # Save the uploaded file temporarily
    file.save(img_path)
    
    predicted_class_index, probabilities = predict_class(img_path, sec_model)
    
    return jsonify({
        'predicted_class_index': predicted_class_index,
        'class_probabilities': probabilities.tolist()
    })

if __name__ == '__main__':
    app.run(debug=True)